## Notebook to test basic AI Platform Jupyter Notebook Functinality
This code executes the following steps:
- Obtain data from BigQuery
- Very basic feature engineering for a simplistic way have handling a categorical variable
- Run a very simple random forest regression model model
- Saves the model objects and related files out to Cloud Storage
- Saves output data to Big Query

### First get needed imports and create variables for the bucket and project name to keep those out of code. 

In [3]:
import os
import pickle

import pandas as pd
from IPython.display import display, HTML
from google.cloud import bigquery
from google.cloud import storage

import modeling.data_prep as dp
import modeling.model_train as mt

bucket_path = os.path.expanduser('~/bucket.txt')
with open(bucket_path) as f:
    BUCKET_NAME = f.read().strip()

    project_path = os.path.expanduser('~/project.txt')
with open(project_path) as f:
    PROJECT_NAME = f.read().strip()

### Pull a small sample of data from a public data set for testing

In [5]:
client = bigquery.Client()

query = """
    SELECT sale_dollars,
           city,
           county_number,
           category,
           store_number,
           item_number,
           date
      FROM `bigquery-public-data.iowa_liquor_sales.sales`
     LIMIT 1000
"""

query_job = client.query(
    query
)

df = query_job.to_dataframe()

df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month
df['day'] = pd.DatetimeIndex(df['date']).day

df = df.drop(['date'], axis=1)

print(f'Length of dataframe: {len(df)}')

SyntaxError: invalid syntax (<ipython-input-5-bf01f34d8b4b>, line 27)

In [8]:
# Categorical Vars to encode
cat_vars = [
    'city',
    'category',
    'county_number',
    'store_number',
    'item_number'
]

# Create dataframe with encoded categorical variables
df_cat, df_mapping = dp.category_columns(
    df,
    cat_vars
)

# Round sales_dollar column
df_cat = df_cat.round({'sale_dollars': 2})

print('Sample of Iowa Liquor Sales Data: ')
display(HTML(df_cat.head().to_html()))


print('Sample of Mapping Data: ')
display(df_mapping.head())

# Save categorical mapping file
df_mapping.to_hdf(
    'categorical_mapping.hdf',
    'df_cat_map',
    format='table',
    mode='w'
)

# Save mapping to storage
storage_client = storage.Client()

bucket = storage_client.bucket(BUCKET_NAME)
blob = bucket.blob('ai_platform_test/iowa_forecasting_testing/categorical_mapping.hdf')
blob.upload_from_filename('categorical_mapping.hdf')

Sample of Iowa Liquor Sales Data: 


,sale_dollars,city,county_number,category,store_number,item_number,year,month,day,city_enc,category_enc,county_number_enc,store_number_enc,item_number_enc
0,813.78,Lemars,75,1701100,2604,13062,2017,5,22,0,0,0,0,0
1,585.00,INDEPENDENCE,10,1701100,3859,2870,2013,10,4,1,0,1,1,1
2,390.24,IOWA CITY,52,1081330,2512,84457,2015,8,19,2,1,2,2,2
3,1558.92,Windsor Heights,77,1081317,3420,82637,2016,6,21,3,2,3,3,3
4,989.40,WEST DES MOINES,77,1081030,2648,67526,2013,2,6,4,3,3,4,4


Sample of Mapping Data: 


,city,category,county_number,store_number,item_number,city_enc,category_enc,county_number_enc,store_number_enc,item_number_enc
0,Lemars,1701100,75,2604,13062,0,0,0,0,0
1,INDEPENDENCE,1701100,10,3859,2870,1,0,1,1,1
2,IOWA CITY,1081330,52,2512,84457,2,1,2,2,2
3,Windsor Heights,1081317,77,3420,82637,3,2,3,3,3
4,WEST DES MOINES,1081030,77,2648,67526,4,3,3,4,4


ImportError: Missing optional dependency 'tables'.  Use pip or conda to install tables.

In [32]:
# Split data into test and train with 3 months of test data
df_train_rfr, df_test_rfr = mt.split_train_test(
    df_cat,
    'month',
    3
)

print('Length of Training Data: ', len(df_train_rfr))
print('Length of Test Data: ', len(df_test_rfr))

display(HTML(df_train_rfr.head().to_html()))
display(HTML(df_test_rfr.head().to_html()))

display(df_train_rfr.describe())

Length of Training Data:  565
Length of Test Data:  435


,sale_dollars,city,county_number,category,store_number,item_number,year,month,day,city_enc,category_enc,county_number_enc,store_number_enc,item_number_enc
0,813.78,Lemars,75,1701100,2604,13062,2017,5,22,0,0,0,0,0
2,390.24,IOWA CITY,52,1081330,2512,84457,2015,8,19,2,1,2,2,2
3,1558.92,Windsor Heights,77,1081317,3420,82637,2016,6,21,3,2,3,3,3
4,989.40,WEST DES MOINES,77,1081030,2648,67526,2013,2,6,4,3,3,4,4
6,3255.60,AMES,85,1011500,4129,27102,2013,1,24,5,4,4,6,5


,sale_dollars,city,county_number,category,store_number,item_number,year,month,day,city_enc,category_enc,county_number_enc,store_number_enc,item_number_enc
1,585.00,INDEPENDENCE,10,1701100,3859,2870,2013,10,4,1,0,1,1,1
5,813.90,WEST DES MOINES,77,1011500,2630,27102,2012,12,13,4,4,3,5,5
7,157.44,Des Moines,77,Unknown,2248,160,2016,11,17,6,5,3,7,6
8,144.00,Boone,8,1701100,5471,100802,2019,12,6,7,0,5,8,7
9,486.00,Waverly,9,1701100,3889,68052,2019,12,10,8,0,6,9,8


,sale_dollars,year,month,day,city_enc,category_enc,county_number_enc,store_number_enc,item_number_enc
count,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000,565.000000
mean,578.727080,2016.074336,5.865487,17.185841,56.088496,12.718584,19.120354,139.047788,99.038938
std,920.865534,2.373344,2.273723,9.742563,54.084541,11.794506,20.279248,118.716165,90.515908
min,0.000000,2012.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,57.150000,2014.000000,5.000000,8.000000,20.000000,4.000000,3.000000,34.000000,23.000000
50%,269.880000,2016.000000,6.000000,18.000000,29.000000,10.000000,14.000000,106.000000,60.000000
75%,648.000000,2019.000000,8.000000,27.000000,82.000000,17.000000,20.000000,221.000000,152.000000
max,8233.200000,2020.000000,9.000000,31.000000,220.000000,49.000000,87.000000,414.000000,334.000000


In [34]:


# Set variable we are predicting for and predictors
y_col = 'sale_dollars'
x_cols = [
    'city_enc',
    'county_number_enc',
    'category_enc',
    'store_number_enc',
    'item_number_enc',
    'year',
    'month',
    'day'
]

# Create model object and importances
rfr_model, importances = mt.fit_model(
    y_col,
    x_cols,
    df_train_rfr,
    trees=150,
    leaves=5
)

# Save model object to shared location on edge node
print('Save Model')
pickle.dump(
    rfr_model,
    open('model_test.pkl', 'wb')
)

print('Model saved')

blob = bucket.blob('ai_platform_test/iowa_forecasting_testing/model_test.pkl')
blob.upload_from_filename('model_test.pkl')

print('Model saved to bucket')

Save Model
Model saved


NameError: name 'bucket' is not defined

In [35]:
df_results = mt.model_predict(
    rfr_model,
    df_test_rfr,
    y_col,
    x_cols
)

# Save results file
df_results.to_hdf(
    'model_test_results.hdf',
    'df_results',
    format='table',
    mode='w'
)

# Save mapping to storage
storage_client = storage.Client()

bucket = storage_client.bucket(BUCKET_NAME)
blob = bucket.blob('ai_platform_test/iowa_forecasting_testing/model_test_results.hdf')
blob.upload_from_filename('model_test_results.hdf')

ImportError: Missing optional dependency 'tables'.  Use pip or conda to install tables.

In [10]:
display(HTML(df_results.head().to_html()))

,sale_dollars,city,county_number,category,store_number,item_number,year,month,day,city_enc,category_enc,county_number_enc,store_number_enc,item_number_enc,predicted_sale_dollars
0,324.54,Mount Vernon,57,1062300,5102,44266,2017,10,26,0,0,0,0,0,1843.578037
1,925.68,BETTENDORF,82,1081317,3952,82637,2012,11,6,1,1,1,1,1,704.884796
6,404.88,Mason City,17,1042100,2515,28088,2018,11,29,6,6,6,6,6,783.086596
7,13.26,Cresco,45,1062100,5244,45247,2018,12,3,7,7,7,7,7,709.673117
8,2740.50,WEST DES MOINES,77,1701100,3899,1400,2012,10,26,8,3,4,8,8,643.111290


In [43]:
#Save results to BigQuery
table_id = 'aiplatform_test.df_results'
project_id = PROJECT_NAME
df_results.to_gbq(table_id, project_id)

#Save dataframe to BigQuery to use in BQ ML example
table_id = 'aiplatform_test.df_cat'
project_id = PROJECT_NAME
df_cat.to_gbq(table_id, project_id)

1it [00:03,  3.68s/it]
1it [00:05,  5.25s/it]


In [44]:
#Imports necessary packages to use BigQuery with Globals
from google.cloud import bigquery
import os
import pandas as pd

# Allow you to easily have Python variables in SQL query.
from IPython.core.magic import register_cell_magic
from IPython import get_ipython

@register_cell_magic('with_globals')
def with_globals(line, cell):
    contents = cell.format(**globals())
    if 'print' in line:
        print(contents)
    get_ipython().run_cell(contents)

In [59]:
%%with_globals
%%bigquery --project $PROJECT_NAME

#Create Model - Dataset being used is the public dataset from above with the encoders 
CREATE or REPLACE MODEL aiplatform_test.liquor_sales_model
OPTIONS
  (model_type='linear_reg',
    input_label_cols=['sale_dollars']) AS
    SELECT sale_dollars,
     city_enc,
     county_number_enc,
     category_enc,
     store_number_enc,
     item_number_enc,
     year,
     month,
     day 
      FROM aiplatform_test.df_cat

""


In [46]:
%%with_globals
%%bigquery --project $PROJECT_NAME

#Evaluate Model 
SELECT
    *
FROM
  ML.EVALUATE(MODEL aiplatform_test.liquor_sales_model,
    (
    SELECT
     sale_dollars,
     city_enc,
     county_number_enc,
     category_enc,
     store_number_enc,
     item_number_enc,
     year,
     month,
     day 
      FROM aiplatform_test.df_cat))

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,669.34761,1.919045e+06,3.535117,483.260347,0.024827,0.025729


In [57]:
%%with_globals
%%bigquery --project $PROJECT_NAME

#Prediction Outcomes
SELECT
    sale_dollars,
     city, 
     county_number,
     category, 
     store_number, 
     item_number, 
     year, 
     month, 
     day, 
     city_enc,
     county_number_enc,
     category_enc,
     store_number_enc,
     item_number_enc,
     predicted_sale_dollars
FROM
  ML.PREDICT(MODEL aiplatform_test.liquor_sales_model,
    (
    SELECT
     sale_dollars,
     city, 
     county_number,
     category,
     store_number, 
     item_number, 
     year, 
     month, 
     day, 
     city_enc,
     county_number_enc,
     category_enc,
     store_number_enc,
     item_number_enc
      FROM aiplatform_test.df_cat
))

,sale_dollars,city,county_number,category,store_number,item_number,year,month,day,city_enc,county_number_enc,category_enc,store_number_enc,item_number_enc,predicted_sale_dollars
0,1125.00,Leclaire,82,1011300,9002,27677,2016,12,22,176,18,18,311,261,522.220582
1,1395.00,Davenport,82,1011300,4196,22178,2019,12,17,24,18,18,49,42,707.423925
2,900.00,Davenport,82,1011300,2635,27677,2019,12,20,24,18,18,354,261,653.252464
3,731.52,WATERLOO,07,1011400,2643,26906,2015,11,9,51,10,26,120,70,910.621410
4,658.26,WINDSOR HEIGHTS,77,1011400,3420,26906,2012,6,27,52,3,26,3,70,999.334463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,90.00,Cedar Rapids,57,Unknown,5207,84197,2019,7,22,25,19,5,316,102,494.242416
996,346.20,COUNCIL BLUFFS,78,Unknown,3443,3345,2012,10,19,107,12,5,52,249,1017.762510
997,327.60,Council Bluffs,78,Unknown,2629,76436,2016,12,5,15,12,5,147,136,977.945825
998,814.20,WEST DES MOINES,77,Unknown,2619,297,2015,11,21,4,3,5,17,16,1012.430267


In [58]:
%%with_globals
%%bigquery --project $PROJECT_NAME

#Create a table with the results 
CREATE TABLE aiplatform_test.bqml_liquor_sales AS (
SELECT
    sale_dollars,
     city, 
     county_number,
     category, 
     store_number, 
     item_number, 
     year, 
     month, 
     day, 
     city_enc,
     county_number_enc,
     category_enc,
     store_number_enc,
     item_number_enc,
     predicted_sale_dollars
FROM
  ML.PREDICT(MODEL aiplatform_test.liquor_sales_model,
    (
    SELECT
     sale_dollars,
     city, 
     county_number,
     category,
     store_number, 
     item_number, 
     year, 
     month, 
     day, 
     city_enc,
     county_number_enc,
     category_enc,
     store_number_enc,
     item_number_enc
      FROM aiplatform_test.df_cat
)))

""
